In [1]:
%load_ext kedro.extras.extensions.ipython

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython


In [2]:
%reload_kedro

2022-08-09 11:19:54,038 - kedro.framework.hooks.manager - INFO - Registered hooks from 1 installed plugin(s): kedro-mlflow-0.8.1
2022-08-09 11:19:54,084 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-08-09 11:19:54,580 - root - INFO - ** Kedro project GetInData ML Framework
2022-08-09 11:19:54,581 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-08-09 11:19:54,591 - root - INFO - Registered line magic `run_viz`
2022-08-09 11:19:54,592 - root - INFO - Registered line magic `reload_kedro_mlflow`


In [3]:
import pandas as pd
import numpy as np

In [4]:
catalog.list()

['articles',
 'customers',
 'transactions',
 'customers_sample',
 'transactions_sample',
 'articles_sample',
 'input_images',
 'resized_images',
 'image_embeddings',
 'text_embeddings',
 'parameters',
 'params:image_embeddings_inference',
 'params:image_embeddings_inference.run_id',
 'params:image_embeddings_inference.image_path',
 'params:image_embeddings_inference.batch_size',
 'params:customers',
 'params:transactions',
 'params:articles',
 'params:image_embeddings',
 'params:image_embeddings.image_path',
 'params:image_embeddings.encoder',
 'params:image_embeddings.decoder',
 'params:image_embeddings.batch_size',
 'params:image_embeddings.image_size',
 'params:image_embeddings.embedding_size',
 'params:image_embeddings.num_epochs',
 'params:image_embeddings.shuffle_reconstructions',
 'params:image_embeddings.save_model',
 'params:image_embeddings.model_name',
 'params:image_embeddings.seed',
 'params:image_resizer',
 'params:image_resizer.output_size',
 'params:image_resizer.method

In [5]:
articles = context.catalog.load('articles_sample')

2022-08-09 11:19:54,619 - kedro.io.data_catalog - INFO - Loading data from `articles_sample` (CSVDataSet)...


In [6]:
customers = context.catalog.load('customers_sample')

2022-08-09 11:19:54,705 - kedro.io.data_catalog - INFO - Loading data from `customers_sample` (CSVDataSet)...


In [7]:
transactions = context.catalog.load('transactions_sample')

2022-08-09 11:19:54,911 - kedro.io.data_catalog - INFO - Loading data from `transactions_sample` (CSVDataSet)...


In [8]:
articles.shape, customers.shape, transactions.shape

((21108, 25), (137198, 7), (157785, 5))

In [9]:
pd.set_option('display.max_columns', None)

### featuretools

In [10]:
import featuretools as ft

In [11]:
es = ft.EntitySet(id="kaggle_hm_data")

In [12]:
import woodwork as ww

ww.list_logical_types()

,name,type_string,description,physical_type,standard_tags,is_default_type,is_registered,parent_type
0,Address,address,Represents Logical Types that contain address ...,string,{},True,True,None
1,Age,age,Represents Logical Types that contain whole nu...,int64,{numeric},True,True,Integer
2,AgeFractional,age_fractional,Represents Logical Types that contain non-nega...,float64,{numeric},True,True,Double
3,AgeNullable,age_nullable,Represents Logical Types that contain whole nu...,Int64,{numeric},True,True,IntegerNullable
4,Boolean,boolean,Represents Logical Types that contain binary v...,bool,{},True,True,BooleanNullable
5,BooleanNullable,boolean_nullable,Represents Logical Types that contain binary v...,boolean,{},True,True,None
6,Categorical,categorical,Represents Logical Types that contain unordere...,category,{category},True,True,None
7,CountryCode,country_code,Represents Logical Types that use the ISO-3166...,category,{category},True,True,Categorical
8,CurrencyCode,currency_code,Represents Logical Types that use the ISO-4217...,category,{category},True,True,Categorical
9,Datetime,datetime,Represents Logical Types that contain date and...,datetime64[ns],{},True,True,None


In [13]:
from woodwork.logical_types import Categorical, Boolean, AgeNullable, Double, NaturalLanguage

In [14]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,ff18c106287931d170618b59fd402ed0ecc292869aad5f...,NaN,NaN,ACTIVE,NONE,31.0,926144af52424f28ab3fb56d84d77b3c81ecadf337126f...
1,136601b1d1f36b13b9a4bbe185f164f24f522c5d476d09...,1.0,1.0,ACTIVE,Regularly,51.0,3c22277b09bfb33d55eae1cf76a6a7084b03082fc10ed4...
2,247a3ee3fe8a153bbf14e41f18c8bb5ebc9d53fe83830d...,NaN,NaN,PRE-CREATE,NONE,69.0,7c5c76766ed0879c1f95a0f922a19cd3458bf642014dba...
3,d1e601dc9ea370b6dee95cbb983adf7fe5294386410be9...,NaN,NaN,ACTIVE,NONE,24.0,39b66dcc48ef29464261279d3e7d911a67867632b851f2...
4,abc428504037054226353c4194380c7f16aff32b0633ac...,NaN,NaN,ACTIVE,NONE,28.0,4fd351ec2879e063a3b2d3f61c86d09311acfab1b2b9de...


In [15]:
es = es.add_dataframe(
    dataframe_name="customers",
    dataframe=customers,
    index="customer_id",
    logical_types={
        "customer_id": Categorical,
        "FN": Boolean,
        "Active": Boolean,
        "club_member_status": Categorical,
        "fashion_news_frequency": Categorical,
        "age": AgeNullable
    },
)

In [16]:
transactions['_index'] = transactions.index

In [17]:
es = es.add_dataframe(
    dataframe_name="transactions",
    dataframe=transactions,
    index="_index",
    time_index="t_dat",
    logical_types={
        "customer_id": Categorical,
        "article_id": Categorical,
        "price": Double,
        "sales_channel_id": Categorical,
    },
)

In [18]:
es = es.add_dataframe(
    dataframe_name="articles",
    dataframe=articles,
    index="article_id",
    logical_types={
        "article_id": Categorical,
        "detail_desc": NaturalLanguage,
    },
)

In [19]:
es

Entityset: kaggle_hm_data
  DataFrames:
    customers [Rows: 137198, Columns: 7]
    transactions [Rows: 157785, Columns: 6]
    articles [Rows: 21108, Columns: 25]
  Relationships:
    No relationships

In [20]:
es = es.add_relationship("customers", "customer_id", "transactions", "customer_id")

In [21]:
es = es.add_relationship("articles", "article_id", "transactions", "article_id")

In [22]:
es

Entityset: kaggle_hm_data
  DataFrames:
    customers [Rows: 137198, Columns: 7]
    transactions [Rows: 157785, Columns: 6]
    articles [Rows: 21108, Columns: 25]
  Relationships:
    transactions.customer_id -> customers.customer_id
    transactions.article_id -> articles.article_id

In [23]:
es['articles'].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
article_id,Categorical,['index']
product_code,Integer,['numeric']
prod_name,Unknown,[]
product_type_no,Integer,['numeric']
product_type_name,Categorical,['category']
product_group_name,Categorical,['category']
graphical_appearance_no,Integer,['numeric']
graphical_appearance_name,Categorical,['category']
colour_group_code,Integer,['numeric']


In [24]:
es['transactions'].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
t_dat,Datetime,['time_index']
customer_id,Categorical,"['foreign_key', 'category']"
article_id,Categorical,"['foreign_key', 'category']"
price,Double,['numeric']
sales_channel_id,Categorical,['category']
_index,Integer,['index']


In [25]:
es['customers'].ww.schema

,Logical Type,Semantic Tag(s)
Column,,
customer_id,Categorical,['index']
FN,Boolean,[]
Active,Boolean,[]
club_member_status,Categorical,['category']
fashion_news_frequency,Categorical,['category']
age,AgeNullable,['numeric']
postal_code,Unknown,[]


### articles

In [26]:
pd.set_option('display.max_rows', None)

In [27]:
ft.primitives.list_primitives()

,name,type,dask_compatible,spark_compatible,description,valid_inputs,return_type
0,avg_time_between,aggregation,False,False,Computes the average number of seconds between...,<ColumnSchema (Logical Type = Datetime) (Seman...,<ColumnSchema (Logical Type = Double) (Semanti...
1,sum,aggregation,True,True,"Calculates the total addition, ignoring `NaN`.",<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
2,num_true,aggregation,True,False,Counts the number of `True` values.,<ColumnSchema (Logical Type = BooleanNullable)...,<ColumnSchema (Logical Type = IntegerNullable)...
3,trend,aggregation,False,False,Calculates the trend of a column over time.,"<ColumnSchema (Semantic Tags = ['numeric'])>, ...",<ColumnSchema (Semantic Tags = ['numeric'])>
4,max,aggregation,True,True,"Calculates the highest value, ignoring `NaN` v...",<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
5,time_since_last,aggregation,False,False,Calculates the time elapsed since the last dat...,<ColumnSchema (Logical Type = Datetime) (Seman...,<ColumnSchema (Logical Type = Double) (Semanti...
6,any,aggregation,True,False,Determines if any value is 'True' in a list.,<ColumnSchema (Logical Type = BooleanNullable)...,<ColumnSchema (Logical Type = Boolean)>
7,mode,aggregation,False,False,Determines the most commonly repeated value.,<ColumnSchema (Semantic Tags = ['category'])>,None
8,count,aggregation,True,True,"Determines the total number of values, excludi...",<ColumnSchema (Semantic Tags = ['index'])>,<ColumnSchema (Logical Type = IntegerNullable)...
9,first,aggregation,False,False,Determines the first value in a list.,<ColumnSchema>,None


In [28]:
pd.set_option('display.max_rows', 30)

In [29]:
articles_feature_matrix, articles_feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="articles",
    agg_primitives=["sum", "mean", "median", "max", "min", "time_since_last", "count", "time_since_first"]
)

In [30]:
articles_feature_matrix

,product_code,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,COUNT(transactions),MAX(transactions.price),MEAN(transactions.price),MEDIAN(transactions.price),MIN(transactions.price),SUM(transactions.price),TIME_SINCE_FIRST(transactions.t_dat),TIME_SINCE_LAST(transactions.t_dat),NUM_CHARACTERS(detail_desc),NUM_WORDS(detail_desc),MAX(transactions.customers.age),MEAN(transactions.customers.age),MEDIAN(transactions.customers.age),MIN(transactions.customers.age),SUM(transactions.customers.age)
article_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0927491001,927491,265,Dress,Garment Full body,1010016,Solid,93,Dark Green,4,Dark,20,Khaki green,1313,Dress,A,Ladieswear,1,Ladieswear,11,Womens Tailoring,1013,Dresses Ladies,60,0.050831,0.048554,0.050831,0.020339,2.913220,6.354480e+07,5.931120e+07,274.0,48.0,66.0,33.016667,28.0,21.0,1981.0
0691839001,691839,272,Trousers,Garment Lower body,1010016,Solid,9,Black,4,Dark,5,Black,1722,Trouser,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1009,Trousers,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,180.0,27.0,NaN,NaN,NaN,NaN,0.0
0908924002,908924,259,Shirt,Garment Upper body,1010016,Solid,9,Black,4,Dark,5,Black,1515,Blouse,A,Ladieswear,1,Ladieswear,11,Womens Tailoring,1010,Blouses,11,0.025407,0.025407,0.025407,0.025407,0.279475,6.337200e+07,5.991600e+07,117.0,19.0,73.0,42.000000,45.0,24.0,462.0
0679630001,679630,262,Jacket,Garment Upper body,1010010,Melange,7,Grey,2,Medium Dusty,12,Grey,1929,Outwear,A,Ladieswear,1,Ladieswear,2,H&M+,1007,Outdoor,1,0.084729,0.084729,0.084729,0.084729,0.084729,6.605040e+07,6.605040e+07,100.0,18.0,52.0,52.000000,52.0,52.0,52.0
0698899003,698899,253,Vest top,Garment Upper body,1010022,Jacquard,9,Black,4,Dark,5,Black,1522,Blouse,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1010,Blouses,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,147.0,27.0,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0475827011,475827,89,Other shoe,Shoes,1010016,Solid,13,Beige,2,Medium Dusty,11,Beige,3630,Shoes / Boots inactive from s5,F,Menswear,3,Menswear,27,Men Shoes,1020,Shoes,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,106.0,17.0,NaN,NaN,NaN,NaN,0.0
0641715002,641715,258,Blouse,Garment Upper body,1010016,Solid,10,White,3,Light,9,White,1545,Projects Woven Tops,D,Divided,2,Divided,50,Divided Projects,1010,Blouses,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,118.0,22.0,NaN,NaN,NaN,NaN,0.0
0545612001,545612,272,Trousers,Garment Lower body,1010023,Denim,72,Blue,2,Medium Dusty,2,Blue,1779,Everyday Waredrobe Denim,A,Ladieswear,1,Ladieswear,57,Ladies Denim,1016,Trousers Denim,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,175.0,29.0,NaN,NaN,NaN,NaN,0.0


In [31]:
ft.describe_feature(articles_feature_defs[30])

'The number of characters in the "detail_desc".'

In [32]:
list(enumerate(articles_feature_defs))

[(0, <Feature: product_code>),
 (1, <Feature: product_type_no>),
 (2, <Feature: product_type_name>),
 (3, <Feature: product_group_name>),
 (4, <Feature: graphical_appearance_no>),
 (5, <Feature: graphical_appearance_name>),
 (6, <Feature: colour_group_code>),
 (7, <Feature: colour_group_name>),
 (8, <Feature: perceived_colour_value_id>),
 (9, <Feature: perceived_colour_value_name>),
 (10, <Feature: perceived_colour_master_id>),
 (11, <Feature: perceived_colour_master_name>),
 (12, <Feature: department_no>),
 (13, <Feature: department_name>),
 (14, <Feature: index_code>),
 (15, <Feature: index_name>),
 (16, <Feature: index_group_no>),
 (17, <Feature: index_group_name>),
 (18, <Feature: section_no>),
 (19, <Feature: section_name>),
 (20, <Feature: garment_group_no>),
 (21, <Feature: garment_group_name>),
 (22, <Feature: COUNT(transactions)>),
 (23, <Feature: MAX(transactions.price)>),
 (24, <Feature: MEAN(transactions.price)>),
 (25, <Feature: MEDIAN(transactions.price)>),
 (26, <Feature

### customers

In [33]:
customers_feature_matrix, customers_feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="customers",
    ignore_dataframes=["articles"],
    agg_primitives=["sum", "mean", "median", "max", "min", "time_since_last", "count", "time_since_first", "avg_time_between"],
)

In [34]:
ft.primitives.list_primitives()

,name,type,dask_compatible,spark_compatible,description,valid_inputs,return_type
0,avg_time_between,aggregation,False,False,Computes the average number of seconds between...,<ColumnSchema (Logical Type = Datetime) (Seman...,<ColumnSchema (Logical Type = Double) (Semanti...
1,sum,aggregation,True,True,"Calculates the total addition, ignoring `NaN`.",<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
2,num_true,aggregation,True,False,Counts the number of `True` values.,<ColumnSchema (Logical Type = BooleanNullable)...,<ColumnSchema (Logical Type = IntegerNullable)...
3,trend,aggregation,False,False,Calculates the trend of a column over time.,"<ColumnSchema (Semantic Tags = ['numeric'])>, ...",<ColumnSchema (Semantic Tags = ['numeric'])>
4,max,aggregation,True,True,"Calculates the highest value, ignoring `NaN` v...",<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
...,...,...,...,...,...,...,...
112,longitude,transform,False,False,Returns the second tuple value in a list of La...,<ColumnSchema (Logical Type = LatLong)>,<ColumnSchema (Semantic Tags = ['numeric'])>
113,add_numeric_scalar,transform,True,True,Adds a scalar to each value in the list.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>
114,tangent,transform,True,True,Computes the tangent of a number.,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Logical Type = Double) (Semanti...
115,modulo_by_feature,transform,True,True,Computes the modulo of a scalar by each elemen...,<ColumnSchema (Semantic Tags = ['numeric'])>,<ColumnSchema (Semantic Tags = ['numeric'])>


In [35]:
customers_feature_matrix

,FN,Active,club_member_status,fashion_news_frequency,age,AVG_TIME_BETWEEN(transactions.t_dat),COUNT(transactions),MAX(transactions.price),MEAN(transactions.price),MEDIAN(transactions.price),MIN(transactions.price),SUM(transactions.price),TIME_SINCE_FIRST(transactions.t_dat),TIME_SINCE_LAST(transactions.t_dat)
customer_id,,,,,,,,,,,,,,
ff18c106287931d170618b59fd402ed0ecc292869aad5f7b2815e43a6fb9dbdf,True,True,ACTIVE,NONE,31,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
136601b1d1f36b13b9a4bbe185f164f24f522c5d476d096bd00a21dd98e20606,True,True,ACTIVE,Regularly,51,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
247a3ee3fe8a153bbf14e41f18c8bb5ebc9d53fe83830d35558463f3df54be98,True,True,PRE-CREATE,NONE,69,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
d1e601dc9ea370b6dee95cbb983adf7fe5294386410be939ed1b91f617d78ae3,True,True,ACTIVE,NONE,24,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
abc428504037054226353c4194380c7f16aff32b0633ac8f9f5d0f5e3da2ae5a,True,True,ACTIVE,NONE,28,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a4092d95d91ec0ff90527962935550346d993a299353c6cf3d00d9fc7b9cc74b,True,True,ACTIVE,NONE,25,NaN,1,0.084729,0.084729,0.084729,0.084729,0.084729,7.088880e+07,7.088880e+07
c321175e12787427aa6d00985519c41650614b362e206f83787c6fc8495d0245,True,True,ACTIVE,Regularly,22,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
016a81548c36c010144819354910dc0763b9b10eeeeacf5145832c28c480c5e0,True,True,ACTIVE,NONE,48,NaN,1,0.050831,0.050831,0.050831,0.050831,0.050831,6.155760e+07,6.155760e+07


In [36]:
list(enumerate(customers_feature_defs))

[(0, <Feature: FN>),
 (1, <Feature: Active>),
 (2, <Feature: club_member_status>),
 (3, <Feature: fashion_news_frequency>),
 (4, <Feature: age>),
 (5, <Feature: AVG_TIME_BETWEEN(transactions.t_dat)>),
 (6, <Feature: COUNT(transactions)>),
 (7, <Feature: MAX(transactions.price)>),
 (8, <Feature: MEAN(transactions.price)>),
 (9, <Feature: MEDIAN(transactions.price)>),
 (10, <Feature: MIN(transactions.price)>),
 (11, <Feature: SUM(transactions.price)>),
 (12, <Feature: TIME_SINCE_FIRST(transactions.t_dat)>),
 (13, <Feature: TIME_SINCE_LAST(transactions.t_dat)>)]

### transactions

transactions features won't be used

In [37]:
transactions_feature_matrix, transactions_feature_defs = ft.dfs(entityset=es, target_dataframe_name="transactions")

In [38]:
transactions_feature_matrix.loc[0, :][0]

'0090852ad6a446ccf02a7d57ec7385c360ab33a813b870e7e725beae6a76ddd4'

In [39]:
transactions[transactions.customer_id=='0090852ad6a446ccf02a7d57ec7385c360ab33a813b870e7e725beae6a76ddd4']

,t_dat,customer_id,article_id,price,sales_channel_id,_index
0,2020-04-01,0090852ad6a446ccf02a7d57ec7385c360ab33a813b870...,0817352006,0.050831,2,0
74757,2020-06-19,0090852ad6a446ccf02a7d57ec7385c360ab33a813b870...,0844030001,0.059305,2,74757
74758,2020-06-19,0090852ad6a446ccf02a7d57ec7385c360ab33a813b870...,0698276009,0.050831,2,74758


In [40]:
transactions_feature_matrix

,customer_id,article_id,price,sales_channel_id,DAY(t_dat),MONTH(t_dat),WEEKDAY(t_dat),YEAR(t_dat),customers.FN,customers.Active,customers.club_member_status,customers.fashion_news_frequency,customers.age,articles.product_code,articles.product_type_no,articles.product_type_name,articles.product_group_name,articles.graphical_appearance_no,articles.graphical_appearance_name,articles.colour_group_code,articles.colour_group_name,articles.perceived_colour_value_id,articles.perceived_colour_value_name,articles.perceived_colour_master_id,articles.perceived_colour_master_name,articles.department_no,articles.department_name,articles.index_code,articles.index_name,articles.index_group_no,articles.index_group_name,articles.section_no,articles.section_name,articles.garment_group_no,articles.garment_group_name,customers.COUNT(transactions),customers.MAX(transactions.price),customers.MEAN(transactions.price),customers.MIN(transactions.price),customers.MODE(transactions.article_id),customers.MODE(transactions.sales_channel_id),customers.NUM_UNIQUE(transactions.article_id),customers.NUM_UNIQUE(transactions.sales_channel_id),customers.SKEW(transactions.price),customers.STD(transactions.price),customers.SUM(transactions.price),articles.COUNT(transactions),articles.MAX(transactions.price),articles.MEAN(transactions.price),articles.MIN(transactions.price),articles.MODE(transactions.customer_id),articles.MODE(transactions.sales_channel_id),articles.NUM_UNIQUE(transactions.customer_id),articles.NUM_UNIQUE(transactions.sales_channel_id),articles.SKEW(transactions.price),articles.STD(transactions.price),articles.SUM(transactions.price),articles.NUM_CHARACTERS(detail_desc),articles.NUM_WORDS(detail_desc)
_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0090852ad6a446ccf02a7d57ec7385c360ab33a813b870...,0817352006,0.050831,2,1,4,2,2020,True,True,ACTIVE,Regularly,61,817352,264,Blazer,Garment Upper body,1010016,Solid,52,Pink,7,Medium,4,Pink,1222,Suit jacket,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1008,Dressed,3,0.059305,0.053655,0.050831,0698276009,2,3,1,1.732051,0.004893,0.160966,41,0.050831,0.045183,0.025407,7c683c445fdd24d6187ebcdc4fb4049356b07bf378a890...,2,38,2,-1.315550,0.007749,1.852492,105.0,17.0
1,02001d047023157327d7a41fbcb9227dfe589ddccfab3c...,0795790001,0.033881,2,1,4,2,2020,True,True,ACTIVE,NONE,24,795790,265,Dress,Garment Full body,1010016,Solid,9,Black,4,Dark,5,Black,1344,Dresses,D,Divided,2,Divided,53,Divided Collection,1013,Dresses Ladies,3,0.033881,0.025407,0.011847,0351484033,2,3,1,-1.574344,0.011864,0.076220,47,0.033881,0.029828,0.020322,7951577d04d50dc98d13b7ac8598fd8ef7832adb801a17...,2,44,2,-0.424343,0.004308,1.401898,140.0,22.0
2,02001d047023157327d7a41fbcb9227dfe589ddccfab3c...,0756347001,0.030492,2,1,4,2,2020,True,True,ACTIVE,NONE,24,756347,265,Dress,Garment Full body,1010001,All over pattern,51,Light Pink,1,Dusty Light,4,Pink,1344,Dresses,D,Divided,2,Divided,53,Divided Collection,1013,Dresses Ladies,3,0.033881,0.025407,0.011847,0351484033,2,3,1,-1.574344,0.011864,0.076220,15,0.050831,0.030140,0.022864,00d950059ced1e3d41312ef4d7c7c26c924881cde6af9c...,2,13,1,1.545844,0.009257,0.452102,220.0,40.0
3,021950aab71ee5b077953803dfa182e04996f69baad474...,0351484002,0.022017,2,1,4,2,2020,True,True,ACTIVE,NONE,27,351484,59,Swimwear bottom,Swimwear,1010016,Solid,9,Black,4,Dark,5,Black,4242,Swimwear,B,Lingeries/Tights,1,Ladieswear,60,"Womens Swimwear, beachwear",1018,Swimwear,3,0.050000,0.029559,0.016661,0351484002,2,3,1,1.559302,0.017904,0.088678,437,0.022017,0.019463,0.003373,974285b4aab6a891e618d15ebd1522e2de7002f89c025f...,2,368,2,-1.559573,0.003223,8.505356,104.0,17.0
4,026a83da294520e421e709d14b3bcc1da4025bd006d820...,0825318005,0.016254,2,1,4,2,2020,True,True,ACTIVE,NONE,25,825318,253,Vest top,Garment Upper body,1010016,Solid,10,White,3,Light,9,White,8316,Ladies Sport Bras,S,Sport,26,Sport,5,Ladies H&M Sport,1005,Jersey Fancy,2,0.024390,0.020322,0.016254,0776237020,2,2,1,NaN,0.005753,0.040644,61,0.01

In [41]:
transactions_feature_defs

[<Feature: customer_id>,
 <Feature: article_id>,
 <Feature: price>,
 <Feature: sales_channel_id>,
 <Feature: DAY(t_dat)>,
 <Feature: MONTH(t_dat)>,
 <Feature: WEEKDAY(t_dat)>,
 <Feature: YEAR(t_dat)>,
 <Feature: customers.FN>,
 <Feature: customers.Active>,
 <Feature: customers.club_member_status>,
 <Feature: customers.fashion_news_frequency>,
 <Feature: customers.age>,
 <Feature: articles.product_code>,
 <Feature: articles.product_type_no>,
 <Feature: articles.product_type_name>,
 <Feature: articles.product_group_name>,
 <Feature: articles.graphical_appearance_no>,
 <Feature: articles.graphical_appearance_name>,
 <Feature: articles.colour_group_code>,
 <Feature: articles.colour_group_name>,
 <Feature: articles.perceived_colour_value_id>,
 <Feature: articles.perceived_colour_value_name>,
 <Feature: articles.perceived_colour_master_id>,
 <Feature: articles.perceived_colour_master_name>,
 <Feature: articles.department_no>,
 <Feature: articles.department_name>,
 <Feature: articles.index_co

### remove

In [42]:
customers_feature_matrix

,FN,Active,club_member_status,fashion_news_frequency,age,AVG_TIME_BETWEEN(transactions.t_dat),COUNT(transactions),MAX(transactions.price),MEAN(transactions.price),MEDIAN(transactions.price),MIN(transactions.price),SUM(transactions.price),TIME_SINCE_FIRST(transactions.t_dat),TIME_SINCE_LAST(transactions.t_dat)
customer_id,,,,,,,,,,,,,,
ff18c106287931d170618b59fd402ed0ecc292869aad5f7b2815e43a6fb9dbdf,True,True,ACTIVE,NONE,31,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
136601b1d1f36b13b9a4bbe185f164f24f522c5d476d096bd00a21dd98e20606,True,True,ACTIVE,Regularly,51,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
247a3ee3fe8a153bbf14e41f18c8bb5ebc9d53fe83830d35558463f3df54be98,True,True,PRE-CREATE,NONE,69,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
d1e601dc9ea370b6dee95cbb983adf7fe5294386410be939ed1b91f617d78ae3,True,True,ACTIVE,NONE,24,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
abc428504037054226353c4194380c7f16aff32b0633ac8f9f5d0f5e3da2ae5a,True,True,ACTIVE,NONE,28,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a4092d95d91ec0ff90527962935550346d993a299353c6cf3d00d9fc7b9cc74b,True,True,ACTIVE,NONE,25,NaN,1,0.084729,0.084729,0.084729,0.084729,0.084729,7.088880e+07,7.088880e+07
c321175e12787427aa6d00985519c41650614b362e206f83787c6fc8495d0245,True,True,ACTIVE,Regularly,22,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
016a81548c36c010144819354910dc0763b9b10eeeeacf5145832c28c480c5e0,True,True,ACTIVE,NONE,48,NaN,1,0.050831,0.050831,0.050831,0.050831,0.050831,6.155760e+07,6.155760e+07


In [43]:
articles_feature_matrix

,product_code,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,COUNT(transactions),MAX(transactions.price),MEAN(transactions.price),MEDIAN(transactions.price),MIN(transactions.price),SUM(transactions.price),TIME_SINCE_FIRST(transactions.t_dat),TIME_SINCE_LAST(transactions.t_dat),NUM_CHARACTERS(detail_desc),NUM_WORDS(detail_desc),MAX(transactions.customers.age),MEAN(transactions.customers.age),MEDIAN(transactions.customers.age),MIN(transactions.customers.age),SUM(transactions.customers.age)
article_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0927491001,927491,265,Dress,Garment Full body,1010016,Solid,93,Dark Green,4,Dark,20,Khaki green,1313,Dress,A,Ladieswear,1,Ladieswear,11,Womens Tailoring,1013,Dresses Ladies,60,0.050831,0.048554,0.050831,0.020339,2.913220,6.354480e+07,5.931120e+07,274.0,48.0,66.0,33.016667,28.0,21.0,1981.0
0691839001,691839,272,Trousers,Garment Lower body,1010016,Solid,9,Black,4,Dark,5,Black,1722,Trouser,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1009,Trousers,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,180.0,27.0,NaN,NaN,NaN,NaN,0.0
0908924002,908924,259,Shirt,Garment Upper body,1010016,Solid,9,Black,4,Dark,5,Black,1515,Blouse,A,Ladieswear,1,Ladieswear,11,Womens Tailoring,1010,Blouses,11,0.025407,0.025407,0.025407,0.025407,0.279475,6.337200e+07,5.991600e+07,117.0,19.0,73.0,42.000000,45.0,24.0,462.0
0679630001,679630,262,Jacket,Garment Upper body,1010010,Melange,7,Grey,2,Medium Dusty,12,Grey,1929,Outwear,A,Ladieswear,1,Ladieswear,2,H&M+,1007,Outdoor,1,0.084729,0.084729,0.084729,0.084729,0.084729,6.605040e+07,6.605040e+07,100.0,18.0,52.0,52.000000,52.0,52.0,52.0
0698899003,698899,253,Vest top,Garment Upper body,1010022,Jacquard,9,Black,4,Dark,5,Black,1522,Blouse,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1010,Blouses,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,147.0,27.0,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0475827011,475827,89,Other shoe,Shoes,1010016,Solid,13,Beige,2,Medium Dusty,11,Beige,3630,Shoes / Boots inactive from s5,F,Menswear,3,Menswear,27,Men Shoes,1020,Shoes,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,106.0,17.0,NaN,NaN,NaN,NaN,0.0
0641715002,641715,258,Blouse,Garment Upper body,1010016,Solid,10,White,3,Light,9,White,1545,Projects Woven Tops,D,Divided,2,Divided,50,Divided Projects,1010,Blouses,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,118.0,22.0,NaN,NaN,NaN,NaN,0.0
0545612001,545612,272,Trousers,Garment Lower body,1010023,Denim,72,Blue,2,Medium Dusty,2,Blue,1779,Everyday Waredrobe Denim,A,Ladieswear,1,Ladieswear,57,Ladies Denim,1016,Trousers Denim,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,175.0,29.0,NaN,NaN,NaN,NaN,0.0


In [44]:
from featuretools.selection import (
    remove_highly_correlated_features,
    remove_highly_null_features,
    remove_single_value_features,
)

In [45]:
remove_highly_null_features(customers_feature_matrix, pct_null_threshold=0.2)

,FN,Active,club_member_status,fashion_news_frequency,age,COUNT(transactions),SUM(transactions.price)
customer_id,,,,,,,
ff18c106287931d170618b59fd402ed0ecc292869aad5f7b2815e43a6fb9dbdf,True,True,ACTIVE,NONE,31,0,0.000000
136601b1d1f36b13b9a4bbe185f164f24f522c5d476d096bd00a21dd98e20606,True,True,ACTIVE,Regularly,51,0,0.000000
247a3ee3fe8a153bbf14e41f18c8bb5ebc9d53fe83830d35558463f3df54be98,True,True,PRE-CREATE,NONE,69,0,0.000000
d1e601dc9ea370b6dee95cbb983adf7fe5294386410be939ed1b91f617d78ae3,True,True,ACTIVE,NONE,24,0,0.000000
abc428504037054226353c4194380c7f16aff32b0633ac8f9f5d0f5e3da2ae5a,True,True,ACTIVE,NONE,28,0,0.000000
...,...,...,...,...,...,...,...
a4092d95d91ec0ff90527962935550346d993a299353c6cf3d00d9fc7b9cc74b,True,True,ACTIVE,NONE,25,1,0.084729
c321175e12787427aa6d00985519c41650614b362e206f83787c6fc8495d0245,True,True,ACTIVE,Regularly,22,0,0.000000
016a81548c36c010144819354910dc0763b9b10eeeeacf5145832c28c480c5e0,True,True,ACTIVE,NONE,48,1,0.050831


In [46]:
new_fm, new_features = remove_single_value_features(customers_feature_matrix, features=customers_feature_defs)

In [47]:
new_fm.shape

(137198, 12)

In [48]:
set(new_features)-set(customers_feature_defs)

set()

In [49]:
customers_feature_matrix

,FN,Active,club_member_status,fashion_news_frequency,age,AVG_TIME_BETWEEN(transactions.t_dat),COUNT(transactions),MAX(transactions.price),MEAN(transactions.price),MEDIAN(transactions.price),MIN(transactions.price),SUM(transactions.price),TIME_SINCE_FIRST(transactions.t_dat),TIME_SINCE_LAST(transactions.t_dat)
customer_id,,,,,,,,,,,,,,
ff18c106287931d170618b59fd402ed0ecc292869aad5f7b2815e43a6fb9dbdf,True,True,ACTIVE,NONE,31,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
136601b1d1f36b13b9a4bbe185f164f24f522c5d476d096bd00a21dd98e20606,True,True,ACTIVE,Regularly,51,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
247a3ee3fe8a153bbf14e41f18c8bb5ebc9d53fe83830d35558463f3df54be98,True,True,PRE-CREATE,NONE,69,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
d1e601dc9ea370b6dee95cbb983adf7fe5294386410be939ed1b91f617d78ae3,True,True,ACTIVE,NONE,24,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
abc428504037054226353c4194380c7f16aff32b0633ac8f9f5d0f5e3da2ae5a,True,True,ACTIVE,NONE,28,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a4092d95d91ec0ff90527962935550346d993a299353c6cf3d00d9fc7b9cc74b,True,True,ACTIVE,NONE,25,NaN,1,0.084729,0.084729,0.084729,0.084729,0.084729,7.088880e+07,7.088880e+07
c321175e12787427aa6d00985519c41650614b362e206f83787c6fc8495d0245,True,True,ACTIVE,Regularly,22,NaN,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
016a81548c36c010144819354910dc0763b9b10eeeeacf5145832c28c480c5e0,True,True,ACTIVE,NONE,48,NaN,1,0.050831,0.050831,0.050831,0.050831,0.050831,6.155760e+07,6.155760e+07


bug in featuretools -> can't call .corr() on Boolean features

https://github.com/alteryx/featuretools/issues/2229

In [50]:
new_fm, new_features = remove_highly_correlated_features(customers_feature_matrix, features=customers_feature_defs)

AttributeError: 'float' object has no attribute 'shape'

In [51]:
list(customers_feature_defs[2:])

[<Feature: club_member_status>,
 <Feature: fashion_news_frequency>,
 <Feature: age>,
 <Feature: AVG_TIME_BETWEEN(transactions.t_dat)>,
 <Feature: COUNT(transactions)>,
 <Feature: MAX(transactions.price)>,
 <Feature: MEAN(transactions.price)>,
 <Feature: MEDIAN(transactions.price)>,
 <Feature: MIN(transactions.price)>,
 <Feature: SUM(transactions.price)>,
 <Feature: TIME_SINCE_FIRST(transactions.t_dat)>,
 <Feature: TIME_SINCE_LAST(transactions.t_dat)>]

In [52]:
new_fm, new_features = remove_highly_correlated_features(customers_feature_matrix.iloc[:, 2:], features=customers_feature_defs[2:])

In [53]:
new_fm

,club_member_status,fashion_news_frequency,age,AVG_TIME_BETWEEN(transactions.t_dat),COUNT(transactions),MAX(transactions.price),MEAN(transactions.price),MIN(transactions.price),SUM(transactions.price),TIME_SINCE_FIRST(transactions.t_dat),TIME_SINCE_LAST(transactions.t_dat)
customer_id,,,,,,,,,,,
ff18c106287931d170618b59fd402ed0ecc292869aad5f7b2815e43a6fb9dbdf,ACTIVE,NONE,31,NaN,0,NaN,NaN,NaN,0.000000,NaN,NaN
136601b1d1f36b13b9a4bbe185f164f24f522c5d476d096bd00a21dd98e20606,ACTIVE,Regularly,51,NaN,0,NaN,NaN,NaN,0.000000,NaN,NaN
247a3ee3fe8a153bbf14e41f18c8bb5ebc9d53fe83830d35558463f3df54be98,PRE-CREATE,NONE,69,NaN,0,NaN,NaN,NaN,0.000000,NaN,NaN
d1e601dc9ea370b6dee95cbb983adf7fe5294386410be939ed1b91f617d78ae3,ACTIVE,NONE,24,NaN,0,NaN,NaN,NaN,0.000000,NaN,NaN
abc428504037054226353c4194380c7f16aff32b0633ac8f9f5d0f5e3da2ae5a,ACTIVE,NONE,28,NaN,0,NaN,NaN,NaN,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
a4092d95d91ec0ff90527962935550346d993a299353c6cf3d00d9fc7b9cc74b,ACTIVE,NONE,25,NaN,1,0.084729,0.084729,0.084729,0.084729,7.088880e+07,7.088880e+07
c321175e12787427aa6d00985519c41650614b362e206f83787c6fc8495d0245,ACTIVE,Regularly,22,NaN,0,NaN,NaN,NaN,0.000000,NaN,NaN
016a81548c36c010144819354910dc0763b9b10eeeeacf5145832c28c480c5e0,ACTIVE,NONE,48,NaN,1,0.050831,0.050831,0.050831,0.050831,6.155760e+07,6.155760e+07
